# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data1_b
　　セッション1のAさんのスケーリングデータ　→　data1_b_label_50

In [2]:
%time
#sensor_dataの読み込み

# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
# data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
# data1_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
# data2_c = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

data1_b_label_50 = pd.read_csv('emotion_label/B/emotion_B_20201111_1_50_es.csv')
data1_b_label_60 = pd.read_csv('emotion_label/B/emotion_B_20201111_1_60_es.csv')
data1_b_label_70 = pd.read_csv('emotion_label/B/emotion_B_20201111_1_70_es.csv')
data1_b_label_80 = pd.read_csv('emotion_label/B/emotion_B_20201111_1_80_es.csv')

# data2_b_label_50 = pd.read_csv('emotion_label/B/emotion_B_20201111_2_50_es.csv')
# data2_b_label_60 = pd.read_csv('emotion_label/B/emotion_B_20201111_2_60_es.csv')
# data2_b_label_70 = pd.read_csv('emotion_label/B/emotion_B_20201111_2_70_es.csv')
# data2_b_label_80 = pd.read_csv('emotion_label/B/emotion_B_20201111_2_80_es.csv')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 8.82 µs


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [3]:
data1_b_label_50

,start_time,finish_time,emotion_level
0,5:36,5:40,0
1,5:37,5:41,0
2,5:38,5:42,0
3,5:39,5:43,0
4,5:40,5:44,0
...,...,...,...
502,20:19,20:23,2
503,20:20,20:24,2
504,20:21,20:25,2
505,20:22,20:26,2


In [4]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

data1_b_label_50 = dropna(data1_b_label_50)
data1_b_label_60 = dropna(data1_b_label_60)
data1_b_label_70 = dropna(data1_b_label_70)
data1_b_label_80 = dropna(data1_b_label_80)

# data2_b_label_50 = dropna(data2_b_label_50)
# data2_b_label_60 = dropna(data2_b_label_60)
# data2_b_label_70 = dropna(data2_b_label_70)
# data2_b_label_80 = dropna(data2_b_label_80)

In [5]:
data1_b_label_50

,start_time,finish_time,emotion_level
0,5:36,5:40,0
1,5:37,5:41,0
2,5:38,5:42,0
3,5:39,5:43,0
4,5:40,5:44,0
...,...,...,...
502,20:19,20:23,2
503,20:20,20:24,2
504,20:21,20:25,2
505,20:22,20:26,2


In [6]:
#生データの確認
display(data1_b)
display(data1_b_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,12:22:31.760,10042,-864,2321,431,111,-73
1,ags,12:22:31.765,10037,-722,2316,217,141,-79
2,ags,12:22:31.770,10169,-725,2353,-87,245,-37
3,ags,12:22:31.775,10120,-791,2397,-154,281,-31
4,ags,12:22:31.780,10169,-591,2468,-270,251,48
...,...,...,...,...,...,...,...,...
456984,ags,13:00:36.915,10288,-1557,3413,-495,592,-591
456985,ags,13:00:36.920,10242,-1103,3222,-660,556,-871
456986,ags,13:00:36.925,10296,-669,3286,-812,415,-1109
456987,ags,13:00:36.930,10371,-289,2971,-763,391,-1140


,start_time,finish_time,emotion_level
0,5:36,5:40,0
1,5:37,5:41,0
2,5:38,5:42,0
3,5:39,5:43,0
4,5:40,5:44,0
...,...,...,...
502,20:19,20:23,2
503,20:20,20:24,2
504,20:21,20:25,2
505,20:22,20:26,2


In [7]:
#合成加速度と合成角速度の算出
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
norm_order(data1_b)
# norm_order(data1_b)
# norm_order(data1_c)
# norm_order(data1_d)
# norm_order(data2_b)
# norm_order(data2_b)
# norm_order(data2_c)
# norm_order(data2_d)

In [8]:
data1_b

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,12:22:31.760,10042,-864,2321,431,111,-73,10342.886493,451.011086
1,ags,12:22:31.765,10037,-722,2316,217,141,-79,10326.011282,270.575313
2,ags,12:22:31.770,10169,-725,2353,-87,245,-37,10462.829206,262.608073
3,ags,12:22:31.775,10120,-791,2397,-154,281,-31,10430.037871,321.928564
4,ags,12:22:31.780,10169,-591,2468,-270,251,48,10480.880974,371.759331
...,...,...,...,...,...,...,...,...,...,...
456984,ags,13:00:36.915,10288,-1557,3413,-495,592,-591,10950.605554,971.992798
456985,ags,13:00:36.920,10242,-1103,3222,-660,556,-871,10793.352445,1226.122751
456986,ags,13:00:36.925,10296,-669,3286,-812,415,-1109,10828.341193,1435.775052
456987,ags,13:00:36.930,10371,-289,2971,-763,391,-1140,10792.034238,1426.411582


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [9]:
#センサデータの時刻合わせ
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

datetime_order(data1_b)
# datetime_order(data1_b)
# datetime_order(data1_c)
# datetime_order(data1_d)

# datetime_order(data2_b)
# datetime_order(data2_b)
# datetime_order(data2_c)
# datetime_order(data2_d)

In [10]:
data1_b

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 12:22:31.760,10042,-864,2321,431,111,-73,10342.886493,451.011086
1,ags,2020-11-11 12:22:31.765,10037,-722,2316,217,141,-79,10326.011282,270.575313
2,ags,2020-11-11 12:22:31.770,10169,-725,2353,-87,245,-37,10462.829206,262.608073
3,ags,2020-11-11 12:22:31.775,10120,-791,2397,-154,281,-31,10430.037871,321.928564
4,ags,2020-11-11 12:22:31.780,10169,-591,2468,-270,251,48,10480.880974,371.759331
...,...,...,...,...,...,...,...,...,...,...
456984,ags,2020-11-11 13:00:36.915,10288,-1557,3413,-495,592,-591,10950.605554,971.992798
456985,ags,2020-11-11 13:00:36.920,10242,-1103,3222,-660,556,-871,10793.352445,1226.122751
456986,ags,2020-11-11 13:00:36.925,10296,-669,3286,-812,415,-1109,10828.341193,1435.775052
456987,ags,2020-11-11 13:00:36.930,10371,-289,2971,-763,391,-1140,10792.034238,1426.411582


In [11]:
data1_b_label_50['start_time'] = pd.to_datetime(data1_b_label_50['start_time'], format='%M:%S')
data1_b_label_50['finish_time'] = pd.to_datetime(data1_b_label_50['finish_time'], format='%M:%S')

In [12]:
data1_b_label_60['start_time'] = pd.to_datetime(data1_b_label_60['start_time'], format='%M:%S')
data1_b_label_60['finish_time'] = pd.to_datetime(data1_b_label_60['finish_time'], format='%M:%S')

In [13]:
data1_b_label_70['start_time'] = pd.to_datetime(data1_b_label_70['start_time'], format='%M:%S')
data1_b_label_70['finish_time'] = pd.to_datetime(data1_b_label_70['finish_time'], format='%M:%S')

In [14]:
data1_b_label_80['start_time'] = pd.to_datetime(data1_b_label_80['start_time'], format='%M:%S')
data1_b_label_80['finish_time'] = pd.to_datetime(data1_b_label_80['finish_time'], format='%M:%S')

In [15]:
# data2_b_label_50['start_time'] = pd.to_datetime(data2_b_label_50['start_time'], format='%M:%S')
# data2_b_label_50['finish_time'] = pd.to_datetime(data2_b_label_50['finish_time'], format='%M:%S')

In [16]:
# data2_b_label_60['start_time'] = pd.to_datetime(data2_b_label_60['start_time'], format='%M:%S')
# data2_b_label_60['finish_time'] = pd.to_datetime(data2_b_label_60['finish_time'], format='%M:%S')

In [17]:
# data2_b_label_70['start_time'] = pd.to_datetime(data2_b_label_70['start_time'], format='%M:%S')
# data2_b_label_70['finish_time'] = pd.to_datetime(data2_b_label_70['finish_time'], format='%M:%S')

In [18]:
# data2_b_label_80['start_time'] = pd.to_datetime(data2_b_label_80['start_time'], format='%M:%S')
# data2_b_label_80['finish_time'] = pd.to_datetime(data2_b_label_80['finish_time'], format='%M:%S')

In [19]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_b)
display(data1_b_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 12:22:31.760,10042,-864,2321,431,111,-73,10342.886493,451.011086
1,ags,2020-11-11 12:22:31.765,10037,-722,2316,217,141,-79,10326.011282,270.575313
2,ags,2020-11-11 12:22:31.770,10169,-725,2353,-87,245,-37,10462.829206,262.608073
3,ags,2020-11-11 12:22:31.775,10120,-791,2397,-154,281,-31,10430.037871,321.928564
4,ags,2020-11-11 12:22:31.780,10169,-591,2468,-270,251,48,10480.880974,371.759331
...,...,...,...,...,...,...,...,...,...,...
456984,ags,2020-11-11 13:00:36.915,10288,-1557,3413,-495,592,-591,10950.605554,971.992798
456985,ags,2020-11-11 13:00:36.920,10242,-1103,3222,-660,556,-871,10793.352445,1226.122751
456986,ags,2020-11-11 13:00:36.925,10296,-669,3286,-812,415,-1109,10828.341193,1435.775052
456987,ags,2020-11-11 13:00:36.930,10371,-289,2971,-763,391,-1140,10792.034238,1426.411582


,start_time,finish_time,emotion_level
0,1900-01-01 00:05:36,1900-01-01 00:05:40,0
1,1900-01-01 00:05:37,1900-01-01 00:05:41,0
2,1900-01-01 00:05:38,1900-01-01 00:05:42,0
3,1900-01-01 00:05:39,1900-01-01 00:05:43,0
4,1900-01-01 00:05:40,1900-01-01 00:05:44,0
...,...,...,...
502,1900-01-01 00:20:19,1900-01-01 00:20:23,2
503,1900-01-01 00:20:20,1900-01-01 00:20:24,2
504,1900-01-01 00:20:21,1900-01-01 00:20:25,2
505,1900-01-01 00:20:22,1900-01-01 00:20:26,2


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [20]:
data1_b_label_50['start_time'] = data1_b_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_b_label_50['finish_time'] = data1_b_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [21]:
data1_b_label_60['start_time'] = data1_b_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_b_label_60['finish_time'] = data1_b_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [22]:
data1_b_label_70['start_time'] = data1_b_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_b_label_70['finish_time'] = data1_b_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [23]:
data1_b_label_80['start_time'] = data1_b_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
data1_b_label_80['finish_time'] = data1_b_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [24]:
# data2_b_label_50['start_time'] = data2_b_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_b_label_50['finish_time'] = data2_b_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [25]:
# data2_b_label_60['start_time'] = data2_b_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_b_label_60['finish_time'] = data2_b_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [26]:
# data2_b_label_70['start_time'] = data2_b_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_b_label_70['finish_time'] = data2_b_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [27]:
# data2_b_label_80['start_time'] = data2_b_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_b_label_80['finish_time'] = data2_b_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [28]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_b_label_50)

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:06,2020-11-11 12:40:10,0
1,2020-11-11 12:40:07,2020-11-11 12:40:11,0
2,2020-11-11 12:40:08,2020-11-11 12:40:12,0
3,2020-11-11 12:40:09,2020-11-11 12:40:13,0
4,2020-11-11 12:40:10,2020-11-11 12:40:14,0
...,...,...,...
502,2020-11-11 12:54:49,2020-11-11 12:54:53,2
503,2020-11-11 12:54:50,2020-11-11 12:54:54,2
504,2020-11-11 12:54:51,2020-11-11 12:54:55,2
505,2020-11-11 12:54:52,2020-11-11 12:54:56,2


In [29]:
#datetime化したラベルデータを保存
# data1_b_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data1_b_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data1_b_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data1_b_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_b_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_b_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_b_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_b_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [30]:
#datetime化した加速度及び角速度データを保存
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data1_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_c.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [31]:
#時刻をインデックスに
data1_b.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data1_c.set_index("時刻", inplace=True)
# data1_d.set_index("時刻", inplace=True)

# data2_b.set_index("時刻", inplace=True)
# data2_b.set_index("時刻", inplace=True)
# data2_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [32]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_b)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 12:22:31.760,ags,10042,-864,2321,431,111,-73,10342.886493,451.011086
2020-11-11 12:22:31.765,ags,10037,-722,2316,217,141,-79,10326.011282,270.575313
2020-11-11 12:22:31.770,ags,10169,-725,2353,-87,245,-37,10462.829206,262.608073
2020-11-11 12:22:31.775,ags,10120,-791,2397,-154,281,-31,10430.037871,321.928564
2020-11-11 12:22:31.780,ags,10169,-591,2468,-270,251,48,10480.880974,371.759331
...,...,...,...,...,...,...,...,...,...
2020-11-11 13:00:36.915,ags,10288,-1557,3413,-495,592,-591,10950.605554,971.992798
2020-11-11 13:00:36.920,ags,10242,-1103,3222,-660,556,-871,10793.352445,1226.122751
2020-11-11 13:00:36.925,ags,10296,-669,3286,-812,415,-1109,10828.341193,1435.775052


In [33]:
data1_b_label_50["start_time"] = data1_b_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_b_label_50["finish_time"] = data1_b_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [34]:
data1_b_label_60["start_time"] = data1_b_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_b_label_60["finish_time"] = data1_b_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [35]:
data1_b_label_70["start_time"] = data1_b_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_b_label_70["finish_time"] = data1_b_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [36]:
data1_b_label_80["start_time"] = data1_b_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data1_b_label_80["finish_time"] = data1_b_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [37]:
# data2_b_label_50["start_time"] = data2_b_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_b_label_50["finish_time"] = data2_b_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [38]:
# data2_b_label_50["start_time"] = data2_b_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_b_label_50["finish_time"] = data2_b_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [39]:
# data2_b_label_50["start_time"] = data2_b_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_b_label_50["finish_time"] = data2_b_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [40]:
# data2_b_label_50["start_time"] = data2_b_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_b_label_50["finish_time"] = data2_b_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [41]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data1_b_label_70)

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:06,2020-11-11 12:40:10,0
1,2020-11-11 12:40:07,2020-11-11 12:40:11,0
2,2020-11-11 12:40:08,2020-11-11 12:40:12,0
3,2020-11-11 12:40:09,2020-11-11 12:40:13,0
4,2020-11-11 12:40:10,2020-11-11 12:40:14,0
...,...,...,...
478,2020-11-11 12:54:49,2020-11-11 12:54:53,2
479,2020-11-11 12:54:50,2020-11-11 12:54:54,2
480,2020-11-11 12:54:51,2020-11-11 12:54:55,2
481,2020-11-11 12:54:52,2020-11-11 12:54:56,2


In [42]:
#startとfinishをリスト化
data1_b_50_start_time = list(data1_b_label_50["start_time"])
data1_b_50_finish_time = list(data1_b_label_50["finish_time"])

data1_b_60_start_time = list(data1_b_label_60["start_time"])
data1_b_60_finish_time = list(data1_b_label_60["finish_time"])

data1_b_70_start_time = list(data1_b_label_70["start_time"])
data1_b_70_finish_time = list(data1_b_label_70["finish_time"])

data1_b_80_start_time = list(data1_b_label_80["start_time"])
data1_b_80_finish_time = list(data1_b_label_80["finish_time"])

# data2_b_50_start_time = list(data2_b_label_50["start_time"])
# data2_b_50_finish_time = list(data2_b_label_50["finish_time"])

# data2_b_60_start_time = list(data2_b_label_60["start_time"])
# data2_b_60_finish_time = list(data2_b_label_60["finish_time"])

# data2_b_70_start_time = list(data2_b_label_70["start_time"])
# data2_b_70_finish_time = list(data2_b_label_70["finish_time"])

# data2_b_80_start_time = list(data2_b_label_80["start_time"])
# data2_b_80_finish_time = list(data2_b_label_80["finish_time"])

In [43]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

data1_b_record_1 = sensor_segmentation_order(data1_b, data1_b_50_start_time, data1_b_50_finish_time, data1_b_label_50)
data1_b_record_2 = sensor_segmentation_order(data1_b, data1_b_60_start_time, data1_b_60_finish_time, data1_b_label_60)
data1_b_record_3 = sensor_segmentation_order(data1_b, data1_b_70_start_time, data1_b_70_finish_time, data1_b_label_70)
data1_b_record_4 = sensor_segmentation_order(data1_b, data1_b_80_start_time, data1_b_80_finish_time, data1_b_label_80)

# data2_b_record_1 = sensor_segmentation_order(data2_b, data2_b_50_start_time, data2_b_50_finish_time, data2_b_label_50)
# data2_b_record_2 = sensor_segmentation_order(data2_b, data2_b_60_start_time, data2_b_60_finish_time, data2_b_label_60)
# data2_b_record_3 = sensor_segmentation_order(data2_b, data2_b_70_start_time, data2_b_70_finish_time, data2_b_label_70)
# data2_b_record_4 = sensor_segmentation_order(data2_b, data2_b_80_start_time, data2_b_80_finish_time, data2_b_label_80)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [44]:
#加速度x,y,z,norm平均値
def acc_gyr_mean_order(data_record, data_label):
    a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]
    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
#     gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
    mean_list = pd.concat([acc_mean_list, gyr_mean_list], axis=1, ignore_index=False)
    return mean_list

data1_b_50_record_mean = acc_gyr_mean_order(data1_b_record_1, data1_b_label_50)
data1_b_60_record_mean = acc_gyr_mean_order(data1_b_record_2, data1_b_label_60)
data1_b_70_record_mean = acc_gyr_mean_order(data1_b_record_3, data1_b_label_70)
data1_b_80_record_mean = acc_gyr_mean_order(data1_b_record_4, data1_b_label_80)

# data2_b_50_record_mean = acc_gyr_mean_order(data2_b_record_1, data2_b_label_50)
# data2_b_60_record_mean = acc_gyr_mean_order(data2_b_record_2, data2_b_label_60)
# data2_b_70_record_mean = acc_gyr_mean_order(data2_b_record_3, data2_b_label_70)
# data2_b_80_record_mean = acc_gyr_mean_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_mean
# b = data1_b_60_record_mean
# c = data1_b_70_record_mean
# d = data1_b_80_record_mean

# e = data2_b_50_record_mean
# f = data2_b_60_record_mean
# g = data2_b_70_record_mean
# h = data2_b_80_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [45]:
#加速度x,y,z,norm分散値
def acc_gyr_var_order(data_record, data_label):
    a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
#     gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
    var_list = pd.concat([acc_var_list, gyr_var_list], axis=1, ignore_index=False)
    return var_list

data1_b_50_record_var = acc_gyr_var_order(data1_b_record_1, data1_b_label_50)
data1_b_60_record_var = acc_gyr_var_order(data1_b_record_2, data1_b_label_60)
data1_b_70_record_var = acc_gyr_var_order(data1_b_record_3, data1_b_label_70)
data1_b_80_record_var = acc_gyr_var_order(data1_b_record_4, data1_b_label_80)

# data2_b_50_record_var = acc_gyr_var_order(data2_b_record_1, data2_b_label_50)
# data2_b_60_record_var = acc_gyr_var_order(data2_b_record_2, data2_b_label_60)
# data2_b_70_record_var = acc_gyr_var_order(data2_b_record_3, data2_b_label_70)
# data2_b_80_record_var = acc_gyr_var_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_var
# b = data1_b_60_record_var
# c = data1_b_70_record_var
# d = data1_b_80_record_var

# e = data2_b_50_record_var
# f = data2_b_60_record_var
# g = data2_b_70_record_var
# h = data2_b_80_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [46]:
#加速度x,y,z,norm分散値
def acc_gyr_max_order(data_record, data_label):
    a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
#     gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
    max_list = pd.concat([acc_max_list, gyr_max_list], axis=1, ignore_index=False)
    return max_list

data1_b_50_record_max = acc_gyr_max_order(data1_b_record_1, data1_b_label_50)
data1_b_60_record_max = acc_gyr_max_order(data1_b_record_2, data1_b_label_60)
data1_b_70_record_max = acc_gyr_max_order(data1_b_record_3, data1_b_label_70)
data1_b_80_record_max = acc_gyr_max_order(data1_b_record_4, data1_b_label_80)

# data2_b_50_record_max = acc_gyr_max_order(data2_b_record_1, data2_b_label_50)
# data2_b_50_record_max = acc_gyr_max_order(data2_b_record_2, data2_b_label_60)
# data2_b_50_record_max = acc_gyr_max_order(data2_b_record_3, data2_b_label_70)
# data2_b_50_record_max = acc_gyr_max_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_max
# b = data1_b_60_record_max
# c = data1_b_70_record_max
# d = data1_b_80_record_max

# e = data2_b_50_record_max
# f = data2_b_60_record_max
# g = data2_b_70_record_max
# h = data2_b_80_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [47]:
#加速度x,y,z,norm分散値
def acc_gyr_min_order(data_record, data_label):
    a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
#     gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
    min_list = pd.concat([acc_min_list, gyr_min_list], axis=1, ignore_index=False)
    return min_list

data1_b_50_record_min = acc_gyr_min_order(data1_b_record_1, data1_b_label_50)
data1_b_60_record_min = acc_gyr_min_order(data1_b_record_2, data1_b_label_60)
data1_b_70_record_min = acc_gyr_min_order(data1_b_record_3, data1_b_label_70)
data1_b_80_record_min = acc_gyr_min_order(data1_b_record_4, data1_b_label_80)

# data2_b_50_record_min = acc_gyr_min_order(data2_b_record_1, data2_b_label_50)
# data2_b_60_record_min = acc_gyr_min_order(data2_b_record_2, data2_b_label_60)
# data2_b_70_record_min = acc_gyr_min_order(data2_c_record_3, data2_b_label_70)
# data2_b_80_record_min = acc_gyr_min_order(data2_d_record_4, data2_b_label_80)

# a = data1_b_50_record_min
# b = data1_b_60_record_min
# c = data1_b_70_record_min
# d = data1_b_80_record_min

# e = data2_b_50_record_min
# f = data2_b_60_record_min
# g = data2_b_70_record_min
# h = data2_b_80_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [48]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成加速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    f = [data_record[i]["合成角速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
    b = pd.concat([data_record_accn_fq_amp_max, data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return b

data1_b_50_record_fq_amp_max = fq_amp_max_order(data1_b_record_1, data1_b_label_50)
data1_b_60_record_fq_amp_max = fq_amp_max_order(data1_b_record_2, data1_b_label_60)
data1_b_70_record_fq_amp_max = fq_amp_max_order(data1_b_record_3, data1_b_label_70)
data1_b_80_record_fq_amp_max = fq_amp_max_order(data1_b_record_4, data1_b_label_80)

# data2_b_50_record_fq_amp_max = fq_amp_max_order(data2_b_record_1, data2_b_label_50)
# data2_b_60_record_fq_amp_max = fq_amp_max_order(data2_b_record_2, data2_b_label_60)
# data2_b_70_record_fq_amp_max = fq_amp_max_order(data2_b_record_3, data2_b_label_70)
# data2_b_80_record_fq_amp_max = fq_amp_max_order(data2_b_record_4, data2_b_label_80)

# a = data1_b_50_record_fq_amp_max
# b = data1_b_60_record_fq_amp_max
# c = data1_b_70_record_fq_amp_max
# d = data1_b_80_record_fq_amp_max

# e = data2_b_50_record_fq_amp_max
# f = data2_b_60_record_fq_amp_max
# g = data2_b_70_record_fq_amp_max
# h = data2_b_80_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

In [49]:
#全特徴量の行列の導出
data1_b_50_record_all_feature = pd.concat([data1_b_50_record_mean, data1_b_50_record_var, data1_b_50_record_max, data1_b_50_record_min, data1_b_50_record_fq_amp_max], axis=1, ignore_index=False)
data1_b_60_record_all_feature = pd.concat([data1_b_60_record_mean, data1_b_60_record_var, data1_b_60_record_max, data1_b_60_record_min, data1_b_60_record_fq_amp_max], axis=1, ignore_index=False)
data1_b_70_record_all_feature = pd.concat([data1_b_70_record_mean, data1_b_70_record_var, data1_b_70_record_max, data1_b_70_record_min, data1_b_70_record_fq_amp_max], axis=1, ignore_index=False)
data1_b_80_record_all_feature = pd.concat([data1_b_80_record_mean, data1_b_80_record_var, data1_b_80_record_max, data1_b_80_record_min, data1_b_80_record_fq_amp_max], axis=1, ignore_index=False)

# data2_b_50_record_all_feature = pd.concat([data2_b_50_record_mean, data2_b_50_record_var, data2_b_50_record_max, data2_b_50_record_min, data2_b_50_record_fq_amp_max], axis=1, ignore_index=False)
# data2_b_60_record_all_feature = pd.concat([data2_b_60_record_mean, data2_b_60_record_var, data2_b_60_record_max, data2_b_60_record_min, data2_b_60_record_fq_amp_max], axis=1, ignore_index=False)
# data2_b_70_record_all_feature = pd.concat([data2_b_70_record_mean, data2_b_70_record_var, data2_b_70_record_max, data2_b_70_record_min, data2_b_70_record_fq_amp_max], axis=1, ignore_index=False)
# data2_b_80_record_all_feature = pd.concat([data2_b_80_record_mean, data2_b_80_record_var, data2_b_80_record_max, data2_b_80_record_min, data2_b_80_record_fq_amp_max], axis=1, ignore_index=False)

In [50]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data1_b_60_record_all_feature

,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,accz_var,gyrx_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,9857.046,-461.974,-2740.522,14.655,158.392,-4.912,12312.335884,6041.021324,131447.137516,86331.813975,...,9597,-705,-3549,-928,-974,-219,7.045010,37.782465,0.782779,217.400421
1,9902.158,-435.735,-2583.761,43.903,89.541,0.794,12551.655036,5674.472775,134190.081879,74971.001591,...,9644,-705,-3549,-928,-974,-219,7.045010,45.442450,0.782779,254.543677
2,9929.489,-429.422,-2485.179,43.122,107.424,-0.560,12093.203879,5856.457916,130518.946959,73191.513116,...,9644,-705,-3549,-928,-974,-219,7.045010,45.617169,0.782779,389.507954
3,9968.190,-412.019,-2355.205,16.598,111.766,9.525,9593.483900,7283.242639,72929.292975,55984.502396,...,9622,-710,-3300,-928,-1315,-176,6.653620,30.442844,0.391389,313.079245
4,9962.997,-425.525,-2369.270,114.003,-47.803,9.002,9311.536991,9327.617375,66632.663100,36174.766991,...,9622,-710,-3054,-751,-1315,-176,7.827789,26.563495,3.131115,72.919865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,9380.835,-110.599,-4041.475,40.930,91.073,-2.528,16437.399775,4710.282199,73741.861375,55566.047100,...,9075,-335,-4499,-611,-876,-133,10.567515,12.948093,0.391389,283.076622
489,9429.997,-103.166,-3929.372,10.457,189.046,14.488,19970.672991,5317.376444,93043.531616,75380.594151,...,9075,-335,-4499,-922,-876,-133,4.305284,52.907536,0.782779,222.835350
490,9478.438,-105.549,-3815.381,76.496,20.565,13.252,19636.888156,5624.065599,93882.021839,69609.639984,...,9075,-335,-4499,-922,-876,-133,3.913894,49.733187,0.782779,194.399429
491,9461.289,-138.199,-3863.885,65.585,49.119,12.442,19421.721479,6713.285399,95387.717775,61592.110775,...,9075,-335,-4499,-922,-876,-109,3.522505,53.293408,0.782779,294.591425


In [51]:
#各特徴量データの保存
data1_b_50_record_all_feature.to_csv('feature/B/data1_b_50_record_all_feature.csv', index=False)
data1_b_60_record_all_feature.to_csv('feature/B/data1_b_60_record_all_feature.csv', index=False)
data1_b_70_record_all_feature.to_csv('feature/B/data1_b_70_record_all_feature.csv', index=False)
data1_b_80_record_all_feature.to_csv('feature/B/data1_b_80_record_all_feature.csv', index=False)

# data2_b_50_record_all_feature.to_csv('feature/B/data2_b_50_record_all_feature.csv', index=False)
# data2_b_60_record_all_feature.to_csv('feature/B/data2_b_60_record_all_feature.csv', index=False)
# data2_b_70_record_all_feature.to_csv('feature/B/data2_b_70_record_all_feature.csv', index=False)
# data2_b_80_record_all_feature.to_csv('feature/B/data2_b_80_record_all_feature.csv', index=False)

In [52]:
data1_b_label_50

,start_time,finish_time,emotion_level
0,2020-11-11 12:40:06,2020-11-11 12:40:10,0
1,2020-11-11 12:40:07,2020-11-11 12:40:11,0
2,2020-11-11 12:40:08,2020-11-11 12:40:12,0
3,2020-11-11 12:40:09,2020-11-11 12:40:13,0
4,2020-11-11 12:40:10,2020-11-11 12:40:14,0
...,...,...,...
502,2020-11-11 12:54:49,2020-11-11 12:54:53,2
503,2020-11-11 12:54:50,2020-11-11 12:54:54,2
504,2020-11-11 12:54:51,2020-11-11 12:54:55,2
505,2020-11-11 12:54:52,2020-11-11 12:54:56,2


In [53]:
data1_b_label_50_time = data1_b_label_50.drop("emotion_level", axis=1)
data1_b_label_60_time = data1_b_label_60.drop("emotion_level", axis=1)
data1_b_label_70_time = data1_b_label_70.drop("emotion_level", axis=1)
data1_b_label_80_time = data1_b_label_80.drop("emotion_level", axis=1)

# data2_b_label_50_time = data2_b_label_50.drop("emotion_level", axis=1)
# data2_b_label_60_time = data2_b_label_60.drop("emotion_level", axis=1)
# data2_b_label_70_time = data2_b_label_70.drop("emotion_level", axis=1)
# data2_b_label_80_time = data2_b_label_80.drop("emotion_level", axis=1)

In [54]:
data1_b_label_50_time

,start_time,finish_time
0,2020-11-11 12:40:06,2020-11-11 12:40:10
1,2020-11-11 12:40:07,2020-11-11 12:40:11
2,2020-11-11 12:40:08,2020-11-11 12:40:12
3,2020-11-11 12:40:09,2020-11-11 12:40:13
4,2020-11-11 12:40:10,2020-11-11 12:40:14
...,...,...
502,2020-11-11 12:54:49,2020-11-11 12:54:53
503,2020-11-11 12:54:50,2020-11-11 12:54:54
504,2020-11-11 12:54:51,2020-11-11 12:54:55
505,2020-11-11 12:54:52,2020-11-11 12:54:56


In [55]:
data1_b_50_record_all_feature = pd.concat([data1_b_label_50_time, data1_b_50_record_all_feature], axis=1)
data1_b_60_record_all_feature = pd.concat([data1_b_label_60_time, data1_b_60_record_all_feature], axis=1)
data1_b_70_record_all_feature = pd.concat([data1_b_label_70_time, data1_b_70_record_all_feature], axis=1)
data1_b_80_record_all_feature = pd.concat([data1_b_label_80_time, data1_b_80_record_all_feature], axis=1)

# data2_b_50_record_all_feature = pd.concat([data2_b_label_50_time, data2_b_50_record_all_feature], axis=1)
# data2_b_60_record_all_feature = pd.concat([data2_b_label_60_time, data2_b_60_record_all_feature], axis=1)
# data2_b_70_record_all_feature = pd.concat([data2_b_label_70_time, data2_b_70_record_all_feature], axis=1)
# data2_b_80_record_all_feature = pd.concat([data2_b_label_80_time, data2_b_80_record_all_feature], axis=1)

In [56]:
data1_b_60_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 12:40:06,2020-11-11 12:40:10,9857.046,-461.974,-2740.522,14.655,158.392,-4.912,12312.335884,6041.021324,...,9597,-705,-3549,-928,-974,-219,7.045010,37.782465,0.782779,217.400421
1,2020-11-11 12:40:07,2020-11-11 12:40:11,9902.158,-435.735,-2583.761,43.903,89.541,0.794,12551.655036,5674.472775,...,9644,-705,-3549,-928,-974,-219,7.045010,45.442450,0.782779,254.543677
2,2020-11-11 12:40:08,2020-11-11 12:40:12,9929.489,-429.422,-2485.179,43.122,107.424,-0.560,12093.203879,5856.457916,...,9644,-705,-3549,-928,-974,-219,7.045010,45.617169,0.782779,389.507954
3,2020-11-11 12:40:09,2020-11-11 12:40:13,9968.190,-412.019,-2355.205,16.598,111.766,9.525,9593.483900,7283.242639,...,9622,-710,-3300,-928,-1315,-176,6.653620,30.442844,0.391389,313.079245
4,2020-11-11 12:40:10,2020-11-11 12:40:14,9962.997,-425.525,-2369.270,114.003,-47.803,9.002,9311.536991,9327.617375,...,9622,-710,-3054,-751,-1315,-176,7.827789,26.563495,3.131115,72.919865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2020-11-11 12:54:49,2020-11-11 12:54:53,9380.835,-110.599,-4041.475,40.930,91.073,-2.528,16437.399775,4710.282199,...,9075,-335,-4499,-611,-876,-133,10.567515,12.948093,0.391389,283.076622
489,2020-11-11 12:54:50,2020-11-11 12:54:54,9429.997,-103.166,-3929.372,10.457,189.046,14.488,19970.672991,5317.376444,...,9075,-335,-4499,-922,-876,-133,4.305284,52.907536,0.782779,222.835350
490,2020-11-11 12:54:51,2020-11-11 12:54:55,9478.438,-105.549,-3815.381,76.496,20.565,13.252,19636.888156,5624.065599,...,9075,-335,-4499,-922,-876,-133,3.913894,49.733187,0.782779,194.399429
491,2020-11-11 12:54:52,2020-11-11 12:54:56,9461.289,-138.199,-3863.885,65.585,49.119,12.442,19421.721479,6713.285399,...,9075,-335,-4499,-922,-876,-109,3.522505,53.293408,0.782779,294.591425


In [57]:
data1_b_50_record_all_feature.to_csv('feature/B/data1_b_50_record_all_feature.csv', index=False)
data1_b_60_record_all_feature.to_csv('feature/B/data1_b_60_record_all_feature.csv', index=False)
data1_b_70_record_all_feature.to_csv('feature/B/data1_b_70_record_all_feature.csv', index=False)
data1_b_80_record_all_feature.to_csv('feature/B/data1_b_80_record_all_feature.csv', index=False)

# data2_b_50_record_all_feature.to_csv('feature/B/data2_b_50_record_all_feature.csv', index=False)
# data2_b_60_record_all_feature.to_csv('feature/B/data2_b_60_record_all_feature.csv', index=False)
# data2_b_70_record_all_feature.to_csv('feature/B/data2_b_70_record_all_feature.csv', index=False)
# data2_b_80_record_all_feature.to_csv('feature/B/data2_b_80_record_all_feature.csv', index=False)

In [58]:
data1_b_60_record_all_feature

,start_time,finish_time,accx_mean,accy_mean,accz_mean,gyrx_mean,gyry_mean,gyrz_mean,accx_var,accy_var,...,accx_min,accy_min,accz_min,gyrx_min,gyry_min,gyrz_min,accn_max_fq,accn_max_peak,gyrn_max_fq,gyrn_max_peak
0,2020-11-11 12:40:06,2020-11-11 12:40:10,9857.046,-461.974,-2740.522,14.655,158.392,-4.912,12312.335884,6041.021324,...,9597,-705,-3549,-928,-974,-219,7.045010,37.782465,0.782779,217.400421
1,2020-11-11 12:40:07,2020-11-11 12:40:11,9902.158,-435.735,-2583.761,43.903,89.541,0.794,12551.655036,5674.472775,...,9644,-705,-3549,-928,-974,-219,7.045010,45.442450,0.782779,254.543677
2,2020-11-11 12:40:08,2020-11-11 12:40:12,9929.489,-429.422,-2485.179,43.122,107.424,-0.560,12093.203879,5856.457916,...,9644,-705,-3549,-928,-974,-219,7.045010,45.617169,0.782779,389.507954
3,2020-11-11 12:40:09,2020-11-11 12:40:13,9968.190,-412.019,-2355.205,16.598,111.766,9.525,9593.483900,7283.242639,...,9622,-710,-3300,-928,-1315,-176,6.653620,30.442844,0.391389,313.079245
4,2020-11-11 12:40:10,2020-11-11 12:40:14,9962.997,-425.525,-2369.270,114.003,-47.803,9.002,9311.536991,9327.617375,...,9622,-710,-3054,-751,-1315,-176,7.827789,26.563495,3.131115,72.919865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2020-11-11 12:54:49,2020-11-11 12:54:53,9380.835,-110.599,-4041.475,40.930,91.073,-2.528,16437.399775,4710.282199,...,9075,-335,-4499,-611,-876,-133,10.567515,12.948093,0.391389,283.076622
489,2020-11-11 12:54:50,2020-11-11 12:54:54,9429.997,-103.166,-3929.372,10.457,189.046,14.488,19970.672991,5317.376444,...,9075,-335,-4499,-922,-876,-133,4.305284,52.907536,0.782779,222.835350
490,2020-11-11 12:54:51,2020-11-11 12:54:55,9478.438,-105.549,-3815.381,76.496,20.565,13.252,19636.888156,5624.065599,...,9075,-335,-4499,-922,-876,-133,3.913894,49.733187,0.782779,194.399429
491,2020-11-11 12:54:52,2020-11-11 12:54:56,9461.289,-138.199,-3863.885,65.585,49.119,12.442,19421.721479,6713.285399,...,9075,-335,-4499,-922,-876,-109,3.522505,53.293408,0.782779,294.591425
